In [ ]:
#Dependencies

import os
from skimage import io

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import matplotlib.patches as patches 
from matplotlib.pyplot import cm

from cellpose import utils
from scipy import signal

from tkinter import Tk
from tkinter.filedialog import askopenfilename, askdirectory
import re
from pathlib import Path
import os.path as path 
import xml.etree.ElementTree as ET 

In [ ]:
#Read delay
try:
    delay = pd.read_csv('delay.txt', header=None).values[0][0] * 1.07 
except OSError:
    delay = 0
    print('delay.txt not found, setting delay = 0')


#Select post seg
post_filename_list = []
for filename in os.listdir():
    if '_seg.npy' in filename:
        post_filename_list.append(filename)
if len(post_filename_list) == 1:
    post_npy_filename = post_filename_list[0]
else:
    #Select POST inputs if more than one `_seg.npy` file in current directory
    Tk().withdraw() 
    post_npy_filename = askopenfilename(title = 'Select POSTtime _seg.npy')

post_npy = np.load(post_npy_filename, allow_pickle=True).item()
print('post_npy_filename = ' + post_npy_filename)


#Select pre seg
pre_npy_filename = path.abspath(path.join(os.getcwd() ,"../..")) + '\\pre\\registered0\\reg_concat_chAvg_sliceAvg_seg.npy'
try:
    pre_dir = os.path.dirname(pre_npy_filename)
    pre_npy = np.load(pre_npy_filename, allow_pickle=True).item()
    print('pre_npy_filename = ' + pre_npy_filename)
except OSError:
    pre_npy_filename = '' #overwrite
    print('No pretime _seg.npy selected; proceeding without pretime')


In [ ]:
#Match somas

if pre_npy_filename == '': #here, create `df` using the number of masks in post_npy, with all indices being equal
    print('No pretime processed, no traces matched') 

    df = pd.DataFrame([])
    df['variable'] = list(range(0, len(np.delete(np.unique(post_npy['masks']), 0)))) #using python indexing, n = # of masks in post_npy, first ROI index = 0
    df['TRACK_ID'] = list(range(0, len(np.delete(np.unique(post_npy['masks']), 0))))
    df['INDEX_0'] = list(range(0, len(np.delete(np.unique(post_npy['masks']), 0))))
    df['INDEX_1'] = list(range(0, len(np.delete(np.unique(post_npy['masks']), 0))))
    df[['INDEX_0','INDEX_1']] = df[['INDEX_0','INDEX_1']].astype(np.int16)

else:
    #Get total number of tracks, for cases where tracks are filtered out
    tree = ET.parse('TrackmateInput.xml')
    root = tree.getroot()
    xml_dict_list = []
    for var in root.iter('Track'):
        xml_dict_list.append(var.attrib)
    FinalTrackIndex = int(xml_dict_list[-1]['TRACK_INDEX']) #0-based indexing, from Trackmate
    
    for csv_name in os.listdir():
        if 'export.csv' in csv_name:
            df_filename = csv_name
            
    df = pd.read_csv(df_filename)
    df = df.iloc[3:]
    df = df[['TRACK_ID','FRAME','MEDIAN_INTENSITY_CH1']] #reads values as str
    df[['TRACK_ID','FRAME','MEDIAN_INTENSITY_CH1']] = df[['TRACK_ID','FRAME','MEDIAN_INTENSITY_CH1']].astype(np.float64)
    df[['TRACK_ID','FRAME','MEDIAN_INTENSITY_CH1']] = df[['TRACK_ID','FRAME','MEDIAN_INTENSITY_CH1']].astype(np.int16)
    df['MEDIAN_INTENSITY_CH1'] = df['MEDIAN_INTENSITY_CH1'] - 1 #subtract 1 to match python indexing
    df = df.rename(columns={'MEDIAN_INTENSITY_CH1': 'INDEX'})

    df.loc[df['FRAME'] == 0, 'INDEX_0'] = df['INDEX']
    df.loc[df['FRAME'] == 1, 'INDEX_1'] = df['INDEX']

    df = df[['TRACK_ID','INDEX_0','INDEX_1']]

    id_vars = ['TRACK_ID']
    melted = df.melt(id_vars=id_vars).dropna()
    pivoted = melted.pivot_table(index=id_vars, columns="variable", values="value")

    df = pivoted.reset_index()
    df[['INDEX_0','INDEX_1']] = df[['INDEX_0','INDEX_1']].astype(np.int16)

    #track indices in CSV (only includes tracks with pre and post spots) are the new matched ROI indices

In [ ]:
#read pre

if pre_npy_filename == '':
    print('No pretime processed')
else:
    now_dir = os.getcwd()
    os.chdir(pre_dir)

    dat = pre_npy
    chan0 = io.imread('reg_concat_ch0.tif') 
    chan1 = io.imread('reg_concat_ch1.tif')

    chan0_list = []
    chan1_list = []

    for n in np.delete(np.unique(dat['masks']), 0):

        ypix_loop = np.where(dat['masks'] == n)[0]
        xpix_loop = np.where(dat['masks'] == n)[1]

        chan0_sig = np.average(chan0[:,ypix_loop ,xpix_loop ], axis = 1) 
        chan1_sig = np.average(chan1[:,ypix_loop ,xpix_loop ], axis = 1) 

        chan0_list.append(chan0_sig)
        chan1_list.append(chan1_sig)
    
    chan0_array = np.vstack(chan0_list)
    chan1_array = np.vstack(chan1_list)

    data_array_pre = np.stack((chan0_array, chan1_array))
    PreFrmsBegin = 0
    PreFrmsCalcd = 120
    data_array_pre = data_array_pre[:,:, PreFrmsBegin : PreFrmsBegin+PreFrmsCalcd ]

    os.chdir(now_dir)

In [ ]:
#filtering pre

if pre_npy_filename == '':
    print('No pretime processed')
else:
    #20 frms moving average boxcar window convolution implementation
    nfrms = 20              
    win = signal.windows.boxcar(nfrms)
    data_array_pre_filt = np.apply_along_axis(lambda m: signal.convolve(m, win, mode='valid') / sum(win) , axis=2, arr=(data_array_pre)) 
    nan_array = np.full([2, np.shape(data_array_pre)[1], int(nfrms/2)], np.nan) 
    data_array_pre_filt = np.concatenate((nan_array, data_array_pre_filt, nan_array), axis = 2)
    
#plotting    
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~    
    plt.rcParams['lines.linewidth'] = 0.5

    fig, axs = plt.subplots(2,3, figsize = (15,10))
    
    axs[0,0].plot(np.transpose(data_array_pre[0]))
    axs[0,0].set_title('CFP Unfiltered')
        
    axs[1,0].plot(np.transpose(data_array_pre_filt[0]))
    axs[1,0].set_title('CFP 20 frms mov avg')
        
    axs[0,1].plot(np.transpose(data_array_pre[1]))
    axs[0,1].set_title('YFP Unfiltered')
    
    axs[1,1].plot(np.transpose(data_array_pre_filt[1]))
    axs[1,1].set_title('YFP 20 frms mov avg')
    
    axs[0,2].plot(np.transpose(data_array_pre[1]/data_array_pre[0]))
    axs[0,2].set_title('FRET Unfiltered')
    
    axs[1,2].plot(np.transpose(data_array_pre_filt[1]/data_array_pre_filt[0]))
    axs[1,2].set_title('FRET 20 frms mov avg')

    plt.show()

    plt.rcParams.update(plt.rcParamsDefault)
    

In [ ]:
#intensity QC pre

if pre_npy_filename == '':
    print('No pretime processed')
else:

    data_array_pre_filt_NaNremoved = data_array_pre_filt[:,:, ~np.isnan(data_array_pre_filt[0,0,:]) ]
    intensity_array_pre = (data_array_pre_filt_NaNremoved[1]+data_array_pre_filt_NaNremoved[0])/2
    print( np.shape(     np.median( intensity_array_pre , axis = 1 )    ) )
    cutoff = 50 

    plt.hist(np.median( intensity_array_pre , axis = 1 ), bins = 110)
    plt.axvline(cutoff, color = 'black')
    plt.show()

    #QC by ROI pixel count
    roiPixelCount_list = []
    for n in np.delete(np.unique(pre_npy['masks']), 0):
        ypix_loop = np.where(pre_npy['masks'] == n)[0]
        roiPixelCount_list.append(len(ypix_loop))
    roiPixelCount_list_PRE = roiPixelCount_list

    passQC_pre = np.intersect1d( np.where(np.median( intensity_array_pre , axis = 1 ) > cutoff)[0]   ,   np.where(np.array(roiPixelCount_list_PRE) > 100) )
    print( np.shape(passQC_pre) )

    inputs = [pre_npy] 
    for dat in inputs:
        outlines = utils.outlines_list(dat['masks'])
        plt.figure(dpi=120)
        plt.imshow(dat['img'])
        for o in outlines:
            plt.plot(o[:,0], o[:,1], color='r', linewidth = 0.3) 
        for o in (outlines[p] for p in passQC_pre):                                            
            plt.plot(o[:,0], o[:,1], color='white', linewidth = 0.3)   
    plt.show()

In [ ]:
#post timelapse

dat = post_npy

chan0 = io.imread('reg_concat_regToPre_ch0.tif') 
chan1 = io.imread('reg_concat_regToPre_ch1.tif')
chan0_list = []
chan1_list = []

for n in np.delete(np.unique(dat['masks']), 0):

    ypix_loop = np.where(dat['masks'] == n)[0]
    xpix_loop = np.where(dat['masks'] == n)[1]

    chan0_sig = np.average(chan0[:,ypix_loop ,xpix_loop ], axis = 1) 
    chan1_sig = np.average(chan1[:,ypix_loop ,xpix_loop ], axis = 1) 
                                                                     
    chan0_list.append(chan0_sig)
    chan1_list.append(chan1_sig)
           
chan0_array = np.vstack(chan0_list)
chan1_array = np.vstack(chan1_list)

data_array = np.stack((chan0_array, chan1_array))

    

In [ ]:
#filtering post

win = signal.windows.boxcar(nfrms) # `nfrms` defined earlier in `filtering pre` 
data_array_filt = np.apply_along_axis(lambda m: signal.convolve(m, win, mode='valid') / sum(win) , axis=2, arr=(data_array))
nan_array = np.full([2, np.shape(data_array)[1], int(nfrms/2)], np.nan) 
data_array_filt = np.concatenate((nan_array, data_array_filt, nan_array), axis = 2)

#plotting    
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~    
plt.rcParams['lines.linewidth'] = 0.5

fig, axs = plt.subplots(2,3, figsize = (15,10))

axs[0,0].plot(np.transpose(data_array[0]))
axs[0,0].set_title('CFP Unfiltered')
    
axs[1,0].plot(np.transpose(data_array_filt[0]))
axs[1,0].set_title('CFP 20 frms mov avg')
    
axs[0,1].plot(np.transpose(data_array[1]))
axs[0,1].set_title('YFP Unfiltered')

axs[1,1].plot(np.transpose(data_array_filt[1]))
axs[1,1].set_title('YFP 20 frms mov avg')

axs[0,2].plot(np.transpose(data_array[1]/data_array[0]))
axs[0,2].set_title('FRET Unfiltered')

axs[1,2].plot(np.transpose(data_array_filt[1]/data_array_filt[0]))
axs[1,2].set_title('FRET 20 frms mov avg')

plt.rcParams.update(plt.rcParamsDefault)

plt.show()


In [ ]:
#intensity QC post

data_array_filt_NaNremoved = data_array_filt[:,:, ~np.isnan(data_array_filt[0,0,:]) ]
intensity_array = (data_array_filt_NaNremoved[1]+data_array_filt_NaNremoved[0])/2
print( np.shape(     np.median( intensity_array , axis = 1 )    ) )
cutoff = 50

plt.hist(np.median( intensity_array , axis = 1 ), bins = 75)
plt.axvline(cutoff, color = 'black')
plt.show()

#QC by ROI pixel count
roiPixelCount_list = []
for n in np.delete(np.unique(post_npy['masks']), 0):
    ypix_loop = np.where(post_npy['masks'] == n)[0]
    roiPixelCount_list.append(len(ypix_loop))
roiPixelCount_list_POST = roiPixelCount_list

passQC_post =  np.intersect1d( np.where(np.median( intensity_array , axis = 1 ) > cutoff)[0]   ,   np.where(np.array(roiPixelCount_list_POST) > 100) )
print( np.shape(passQC_post) )

inputs = [post_npy]
for dat in inputs:
    outlines = utils.outlines_list(dat['masks'])
    plt.figure(dpi=120)
    plt.imshow(dat['img'])
    for o in outlines:
        plt.plot(o[:,0], o[:,1], color='r', linewidth = 0.3) 
    for o in (outlines[p] for p in passQC_post):                                            
        plt.plot(o[:,0], o[:,1], color='white', linewidth = 0.3)   
plt.show()

In [ ]:
#create masked arrays

if pre_npy_filename == '':
    print("no pretime processed")

    mask_post = np.ones(np.shape(    data_array_filt    ), bool)
    mask_post[:, passQC_post, :] = False
    data_array_filt_masked = np.ma.MaskedArray(data_array_filt, mask_post)

else:

    mask_pre = np.ones(np.shape(     data_array_pre_filt    ), bool) 
    mask_pre[:, passQC_pre, :] = False 
    data_array_pre_filt_masked = np.ma.MaskedArray(data_array_pre_filt, mask_pre) #Non Matched, QC'd
    
    mask_post = np.ones(np.shape(    data_array_filt    ), bool)
    mask_post[:, passQC_post, :] = False
    data_array_filt_masked = np.ma.MaskedArray(data_array_filt, mask_post)

    #Matched, QC'd
    MATCHED_data_array_pre_filt_masked = data_array_pre_filt_masked[:,df.INDEX_0.values,:] 
    MATCHED_data_array_filt_masked = data_array_filt_masked[:,df.INDEX_1.values,:]
    mask_union_index = np.union1d(np.where(MATCHED_data_array_pre_filt_masked.mask.all(axis=0).all(axis=1))[0], 
            np.where(MATCHED_data_array_filt_masked.mask.all(axis=0).all(axis=1))[0])
    MATCHED_data_array_pre_filt_masked[:,mask_union_index,:] = np.ma.masked  #overwrite
    MATCHED_data_array_filt_masked[:,mask_union_index,:] = np.ma.masked
   


In [ ]:
#save_csv

data_array_filt_masked #filt, QC'd, nonMATCHED
unmatched_PostFRET = data_array_filt_masked[1]/data_array_filt_masked[0]
pd.DataFrame(unmatched_PostFRET).to_csv('Unmatched_FRET_POST.csv', header=False, index=False)

MATCHED_data_array_filt_masked #filt, QC'd, MATCHED
PostFRET = MATCHED_data_array_filt_masked[1]/MATCHED_data_array_filt_masked[0]
pd.DataFrame(PostFRET).to_csv('Matched_FRET_POST.csv', header=False, index=False)

data_array_pre_filt_masked  
unmatched_PreFRET = data_array_pre_filt_masked[1]/data_array_pre_filt_masked[0]
pd.DataFrame(unmatched_PreFRET).to_csv('Unmatched_FRET_PRE.csv', header=False, index=False)

MATCHED_data_array_pre_filt_masked
PreFRET = MATCHED_data_array_pre_filt_masked[1]/MATCHED_data_array_pre_filt_masked[0]
pd.DataFrame(PreFRET).to_csv('Matched_FRET_PRE.csv', header=False, index=False)